### これまでの結果とoriginal_textを見て検討するnotebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from config import *
from bert_utils import *

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

****** SEED fixed : 42 ******




### データ読み込み --

In [2]:
train = {
    "raw": pd.read_feather("input/dataset_with_original_text/train_with_original_text.feather"),
    "best": pd.read_feather("/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/roberta_large_cat4_fold8/train_df.feather"),
    "org1": pd.read_feather("output/roberta_large_cat4_org_batch16_acc2_fold8/train_df.feather"),
}

In [3]:
test = {
    "raw": pd.read_feather("input/dataset_with_original_text/test_with_original_text.feather"),
    "best": pd.read_feather("/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/roberta_large_cat4_fold8/test_df.feather"),
    "org1": pd.read_feather("output/roberta_large_cat4_org_batch16_acc2_fold8/test_df.feather"),
}

### まず、max_len == 76になってない？ --
* というより、original_textがclean_textに入ってない気がしますね

In [4]:
i = 10
print(repr(train["org1"].loc[i, "clean_text"]))
print(repr(train["org1"].loc[i, "text"]))
print(repr(train["org1"].loc[i, "original_text"]))

'女 は 精子 が なくて も 男 は 卵子 が なくて も 産める と は おもう 現 段階 でも 倫理 観 が 人間 の 進歩 を 疎外 して る の か な'
'女は精子がなくても\n男は卵子がなくても産めるとはおもう現段階でも\n倫理観が人間の進歩を疎外してるのかな'
'女は精子がなくても __BR__ 男は卵子がなくても産めるとはおもう現段階でも __BR__ 倫理観が人間の進歩を疎外してるのかな\tいやそれ人類の進歩じゃねーだろ __BR__ それに医療倫理は私達自身をまもってるんだよ？\t人間の体にメスを入れた人間はそんなこと気にして無かったと思うわ\n'


### tokenizeまでを追ってみる --

In [5]:
df, train_shape = prepare_dataframe(train_data="raw_original_text")
df = df.head(100)
df["label"] = 0
train_shape = 20

In [6]:
train_df, test_df = preprocess_text(df, train_shape, "nlp-waseda/roberta-large-japanese-seq512", train_data="raw_original_text")

... ... Process -> raw_original_text


In [9]:
train_df["clean_text"][10]

'女 は 精子 が なくて も 男 は 卵子 が なくて も 産める と は おもう 現 段階 でも 倫理 観 が 人間 の 進歩 を 疎外 して る の か な [SEP] いや それ 人類 の 進歩 じゃ ねー だろ それ に 医療 倫理 は 私 達 自身 を まもって る んだ よ ? [SEP] 人間 の 体 に メス を 入れた 人間 は そんな こと 気 に して 無かった と 思う わ'

* original_textの処理がうまいこといってなかったです
* とりあえずそれ使ってサブミットしてみてかな〜


* やってみた

In [6]:
import pandas as pd
tt = pd.read_feather("/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/roberta_large_cat4_org_batch16_acc2_fold8/train_df.feather")

In [8]:
# ok --
tt["clean_text"][10]

'女 は 精子 が なくて も 男 は 卵子 が なくて も 産める と は おもう 現 段階 でも 倫理 観 が 人間 の 進歩 を 疎外 して る の か な [SEP] いや それ 人類 の 進歩 じゃ ねー だろ それ に 医療 倫理 は 私 達 自身 を まもって る んだ よ ? [SEP] 人間 の 体 に メス を 入れた 人間 は そんな こと 気 に して 無かった と 思う わ'